In [ ]:
#default_exp selection

# Data Selection

In [ ]:
#export
import ipywidgets as widgets
import traitlets
from traitlets import Int, List, observe
from template_lib.data import DataModel, FILENAME, DataDelegate
from IPython.display import clear_output, FileLink

import logging
logger = logging.getLogger()

In [ ]:
#export
CRITERIA_TITLE = 'Selection Criteria'
CRITERIA_APPLY = 'Filter'
OUTPUT_TITLE = 'Results'
OUTPUT_PRE = 'Limit to '
OUTPUT_POST = 'lines'
EXPORT_TITLE = 'Export'
EXPORT_BUTTON = 'Download'
START_YEAR = 'From Year'
END_YEAR = 'To Year'
EXPORT_LINK_PROMPT = "Click here to save file: "

In [ ]:
data = DataModel(FILENAME)
data._trait_values

In [ ]:
#export
class FilterView(widgets.Accordion):
    
    def __init__(self):
        super().__init__()
        
        self.set_title(0, 'Selection Criteria')
        self.lo20 = widgets.Layout(width='20%')
        
        self.start_year = widgets.Dropdown(description= START_YEAR)
        self.end_year = widgets.Dropdown(description= END_YEAR)
        
        self.filter_btn_apply = widgets.Button(description= CRITERIA_APPLY, icon='filter',
                                               layout=self.lo20)
        self.invalid = widgets.HTML(value = f"<b><font color='red'>End year must be greater or equal to start year.</b>")
        hbox = widgets.HBox(children=(self.filter_btn_apply, self.invalid))
        
        self.vbox = widgets.VBox(children = (self.start_year, self.end_year, hbox))
        self.children = (self.vbox, )

In [ ]:
filterView = FilterView()
filterView

In [ ]:
#export
class FilterDelegate(FilterView):
    
    def __init__(self, model=None):
        super().__init__()
        if model:
            self.initialize(model)
        
    def initialize(self, model):
        self.model = model
        self.start_year.options = [i for i in range(self.model.min_year, self.model.max_year +1)]
        self.end_year.options = [i for i in range(self.model.min_year, self.model.max_year + 1)]
        traitlets.link((self.model, 'end_year'), (self.end_year, 'value'))
        traitlets.link((self.model, 'start_year'), (self.start_year, 'value'))
        self.invalid.layout.visibility = 'hidden'
        
        # the 'value' parameter below is important. If you don't specify it,
        # you may get a KeyError because you are accidentally observing events
        # of type _property_lock
        self.start_year.observe(self.observe_start_year, 'value')
        self.end_year.observe(self.observe_end_year, 'value')
        self.filter_btn_apply.on_click(self.on_button_click)
        
    def observe_start_year(self, change):
        self.model.start_year = change['new']
        self.valid_years()
        
    def observe_end_year(self, change):
        self.model.end_year = change['new']
        self.valid_years()
    
    def on_button_click(self, btn):
        if self.valid_years():
            self.model.filter_data()
            
    def valid_years(self):
        if self.model.start_year > self.model.end_year:
            self.invalid.layout.visibility = 'visible'
            return False
        else:
            self.invalid.layout.visibility = 'hidden'
            return True

In [ ]:
dataModel = DataModel(FILENAME)
filterDelegate = FilterDelegate(dataModel)
filterDelegate

In [ ]:
#export
class ResultsDelegate(widgets.Accordion):
    
    def __init__(self, model=None):
        super().__init__()
        self.set_title(0, 'Results')
        if model:
            self.initialize(model)
        
    def initialize(self, model):
        self.model = model
        self.data = DataDelegate(model)
        self.children = (self.data, )

In [ ]:
results = ResultsDelegate(dataModel)
results

In [ ]:
#export
class ExportView(widgets.Accordion):
    
    def __init__(self):
        super().__init__()
        self.set_title(0, 'Export')
        self.lo20 = widgets.Layout(width='20%')
                       
        ## child widgets
        self.filter_output = widgets.Output()
        self.filter_btn_refexp = widgets.Button(description= EXPORT_BUTTON, icon='download', layout=self.lo20)
        self.filter_out_export = widgets.Output(layout={'border': '1px solid black'})

        vbox = widgets.VBox(children=(self.filter_btn_refexp, self.filter_out_export))
        self.children = (vbox, )

In [ ]:
exportView = ExportView()
exportView

In [ ]:
#export
class ExportDelegate(ExportView):
    
    def __init__(self, model = None):
        super().__init__()
        if model:
            self.initialize(model)
            
        self.filter_btn_refexp.on_click(self.observe_download_btn)
        
    def initialize(self, model):
        self.model = model
    
    def observe_download_btn(self, btn):
        filename = self.model.create_download_file()
        with self.filter_out_export:
            clear_output(wait=True)
            display(FileLink(filename, result_html_prefix=EXPORT_LINK_PROMPT))

In [ ]:
exportDelegate = ExportDelegate(dataModel)
exportDelegate

In [ ]:
#export
class SelectionTab(widgets.VBox):
    
    def __init__(self, model):
        super().__init__()
        self.model = model
        
        self.filter_acc = FilterDelegate(model)
        self.filter_acc.filter_btn_apply.on_click(self.observe_filter)
        
        self.results_acc = ResultsDelegate(model)
        self.export_acc = ExportDelegate(model)
        # Add all accordions to section tab (self)
        self.children = (self.filter_acc, self.results_acc, self.export_acc)
    
    def observe_filter(self, btn):
        self.results_acc.data.display_data()

In [ ]:
selectionTab = SelectionTab(dataModel)
selectionTab

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()